In [ ]:
%matplotlib inline


# 3D map of the galaxy

Create inferred map of the galaxy from chirping binaries


This example demonstrates using chirping binaries to map the galaxy.
Samples from the high SNR chirping binaries are reparameterized into galactic cartesian coordinates and plotted.



In [ ]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.colors as colors
import matplotlib.cm as cm
from lisacattools.catalog import GWCatalogs, GWCatalogType
from lisacattools import (
    get_DL,
    confidence_ellipse,
    convert_ecliptic_to_galactic,
    convert_galactic_to_cartesian,
)

# Start by loading the main catalog file processed from GBMCMC outputs
catPath = "../../tutorial/data/ucb"
catName = "cat15728640_v2.h5"
catalogs = GWCatalogs.create(GWCatalogType.UCB, catPath, catName)
catalog = catalogs.get_last_catalog()

# Get SNRs and fdots for all detections to use for event selection
detections = catalog.get_detections(["SNR","Frequency Derivative"])

# Get dataframe of only high SNR chirping events
selected_detections = detections[
    (detections["Frequency Derivative"] > 0) & (detections["SNR"] > 100)
]

# Sort them so the highest chirp rates are plotted last
selected_detections=selected_detections.sort_values(by='Frequency Derivative')

plot 1-sigma ellipses of 3D localization for each source




In [ ]:
# set up the figure
fig, axs = plt.subplots(1, 2, figsize=(12, 6), dpi=100)

axs[0].grid()
axs[1].grid()

axs[0].set(
    xlim=(-20, 20),
    ylim=(-20, 20),
    xlabel=r"$x_{\rm GC}\ [{\rm kpc}]$",
    ylabel=r"$y_{\rm GC}\ [{\rm kpc}]$",
)
axs[1].set(
    xlim=(-20, 20),
    ylim=(-20, 20),
    xlabel=r"$x_{\rm GC}\ [{\rm kpc}]$",
    ylabel=r"$z_{\rm GC}\ [{\rm kpc}]$",
)


# color ellipses by log SNR
cNorm = colors.LogNorm(
    vmin=selected_detections["SNR"].min(), vmax=selected_detections["SNR"].max()
)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=plt.cm.get_cmap("plasma_r"))
cbar = fig.colorbar(scalarMap)
cbar.set_label("SNR")

# get list of all sources in catalog
sources = list(selected_detections.index)

# git list of all chain parameters (we need most of them)
parameters = catalog.get_attr_source_sample(sources[0])

for source in sources:

    # get chain samples
    samples = catalog.get_source_sample(source,parameters)

    # fix Ecliptic Latitude bug in HDF5 files
    samples["Ecliptic Latitude"] = np.pi/2 - np.arccos(samples["coslat"])

    # convert from ecliptic to galactic coordinates
    convert_ecliptic_to_galactic(samples)
    
    # enforce GR prior
    samples = samples[(samples["Frequency Derivative"] > 0)]
    
    # add distance parameter
    get_DL(samples)

    # add galactic cartesian coordinates
    convert_galactic_to_cartesian(
        samples, "Galactic Longitude", "Galactic Latitude", "Luminosity Distance"
    )

    # plot galactic X-Y plane
    confidence_ellipse(
        samples[["X", "Y"]],
        axs[0],
        n_std=1.0,
        edgecolor=scalarMap.to_rgba(np.array(detections.loc[source].SNR)),
        linewidth=1.0,
    )

    # plot galactic X-Z plane
    confidence_ellipse(
        samples[["X", "Z"]],
        axs[1],
        n_std=1.0,
        edgecolor=scalarMap.to_rgba(np.array(detections.loc[source].SNR)),
        linewidth=1.0,
    )